## Import libraries

In [1]:
suppressPackageStartupMessages({
library(dplyr)
library(Seurat)
library(readr)
library(tidyr)
library(SCopeLoomR)
})

Warning message:
"package 'SCopeLoomR' was built under R version 4.0.5"


In [2]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /vsc-hard-mounts/leuven-data/337/vsc33726/miniconda3/envs/R_scAnalysis/lib/libopenblasp-r0.3.12.so

locale:
[1] C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] SCopeLoomR_0.3.1   tidyr_1.1.3        readr_1.4.0        SeuratObject_4.0.0
[5] Seurat_4.0.1       dplyr_1.0.5       

loaded via a namespace (and not attached):
  [1] Rtsne_0.15            colorspace_2.0-0      deldir_0.2-10        
  [4] ellipsis_0.3.1        ggridges_0.5.3        IRdisplay_1.0        
  [7] base64enc_0.1-3       spatstat.data_2.1-0   leiden_0.3.7         
 [10] listenv_0.8.0         ggrepel_0.9.1         bit64_4.0.5          
 [13] fansi_0.4.2           codetools_0.2-18      splines_4.0.3        
 [16] polyclip_1.10-0       IRkernel_1.1.1        jsonlite_1.7.2       
 [19] ica_1.0-2 

# Set working directory

In [2]:
setwd("/staging/leuven/stg_00041/Ryan/Irene_NCB")

# Import Seurat object

In [3]:
seurat <- readRDS("d4IT_Zhou_Pet_PRC2i_split.rds")

In [5]:
seurat@meta.data %>% colnames()
seurat@meta.data %>% head(1)

[1] "orig.ident.x"    "nCount_RNA"      "nFeature_RNA"    "percent.mt"     
 [5] "percent.ERCC"    "Sample"          "orig.ident.y"    "Experiment"     
 [9] "combine_state"   "final_lineage"   "Cell.type"       "finalClusters"  
[13] "Lineage"         "RNA_snn_res.0.1" "seurat_clusters" "percent.MT"     
[17] "RNA_snn_res.0.2"

,orig.ident.x,nCount_RNA,nFeature_RNA,percent.mt,percent.ERCC,Sample,orig.ident.y,Experiment,combine_state,final_lineage,Cell.type,finalClusters,Lineage,RNA_snn_res.0.1,seurat_clusters,percent.MT,RNA_snn_res.0.2
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
D10_IVC4_E1_B1_10,VP_Messmer_Petropoulos_Zhou,788590,10072,0,0,D10_IVC4_E1_B1_10,IVC4,Zhou,E1,TE,post_implantation,TB.medium2,TE _ Zhou,0,0,NA,NA


## Remove the ".1" from the cell names

In [19]:
seurat <- RenameCells(seurat, new.names = gsub(".1", "", rownames(seurat@meta.data)))

## Change the "sample" column of metadata

In [ ]:
seurat@meta.data[grep("TSC[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample <- gsub("TSC", "Day 4 Trophoblast", seurat@meta.data[grep("TSC[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample)
seurat@meta.data[grep("TSCP", rownames(seurat@meta.data), perl=TRUE),]$sample <- gsub("TSCPRC2i", "Day 4 Trophoblast PRC2i", seurat@meta.data[grep("TSCP", rownames(seurat@meta.data), perl=TRUE),]$sample)
seurat@meta.data[grep("NAIVE[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample <-  gsub("NAIVE", "Day 0 Naive hPSCs", seurat@meta.data[grep("NAIVE[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample)
seurat@meta.data[grep("NAIVEP", rownames(seurat@meta.data), perl=TRUE),]$sample <- gsub("NAIVEPRC2i", "Day 0 Naive hPSCs PRC2i", seurat@meta.data[grep("NAIVEP", rownames(seurat@meta.data), perl=TRUE),]$sample)
head(seurat@meta.data)

### Check the sample column to make sure it only contains the names you want

In [50]:
seurat@meta.data$sample %>% unique()

[1] "Day 4 Trophoblast"       "Day 4 Trophoblast PRC2i"
[3] "Day 0 Naive hPSCs"       "Day 0 Naive hPSCs PRC2i"

# Create loom file
#### This will fail if a loom file of the same name already exists

In [6]:
build_loom(file.name = "Irene_integrated.loom",
        dgem = seurat@assays$RNA@counts,
        title = "Naive to trophoblast conversion with(out) PRC2 inhibition, integrated with embryo data",
        default.embedding = seurat@reductions$umap@cell.embeddings,
        default.embedding.name = "UMAP")

[1] "Adding global attributes..."
[1] "Adding matrix..."
  |======================================================================| 100%[1] "Adding column attributes..."
[1] "Adding default metrics nUMI..."
[1] "Adding default metrics nGene..."
[1] "Adding default embedding..."
[1] "Adding row attributes..."
[1] "Adding columns edges..."
[1] "Adding row edges..."
[1] "Adding layers..."


## Open the loom file for editing

In [9]:
#system("cp Irene.loom Irene.loom_bk")
#system("cp Irene.loom_bk Irene.loom")


#loom <- open_loom("Irene.loom")
loom <- open_loom("Irene_scale_data.loom")

## Add two attribute columns
#### Key:

* name of the attribute

#### as.annotation = TRUE:

* discrete attribute

#### as.metric = TRUE:

* continuous attribute

In [10]:
add_col_attr(loom = loom, key = "cluster",
              value = seurat@meta.data$seurat_clusters, as.annotation = TRUE)
add_col_attr(loom = loom, key = "annotation",
              value = seurat@meta.data$sample, as.annotation = TRUE)

# Done!
#### Loom file is automatically updated

# Create LOOM fle for integration of d4 trophoblast +/- PRC2i with human pre and post-implantation data sets.

## Import libraries

In [1]:
suppressPackageStartupMessages({
library(dplyr)
library(Seurat)
library(readr)
library(tidyr)
library(SCopeLoomR)
})

Warning message:
"package 'SCopeLoomR' was built under R version 4.0.5"


# Set working directory

In [2]:
setwd("/staging/leuven/stg_00041/Ryan/Irene_NCB")

# Import Seurat object

In [3]:
seurat <- readRDS("d4IT_Zhou_Pet_PRC2i_split.rds")

In [4]:
data_IT <- readRDS("seurat_object_norm.rds")

In [5]:
data_IT@meta.data %>% head()

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,sample,RNA_snn_res.0.2,seurat_clusters
,<chr>,<dbl>,<int>,<dbl>,<chr>,<fct>,<fct>
TSC_AAACCCACACTCCGGA.1,naive_TSC_PRC2i_Irene_VP,79984,8416,12.40998,TSC,1,1
TSC_AAACGAAGTAAGGCCA.1,naive_TSC_PRC2i_Irene_VP,187905,10316,13.33493,TSC,1,1
TSC_AAACGCTTCTACTGAG.1,naive_TSC_PRC2i_Irene_VP,103178,8633,12.52302,TSC,2,2
TSC_AAAGGGCCATGGAATA.1,naive_TSC_PRC2i_Irene_VP,70210,8112,18.62555,TSC,1,1
TSC_AAAGGTAAGATGCTGG.1,naive_TSC_PRC2i_Irene_VP,67006,7910,15.51354,TSC,1,1
TSC_AAAGTCCAGAAACACT.1,naive_TSC_PRC2i_Irene_VP,56056,7707,16.82068,TSC,1,1


## Remove the ".1" from the cell names

In [19]:
#seurat <- RenameCells(seurat, new.names = gsub(".1", "", rownames(seurat@meta.data)))

## Change the "sample" column of metadata

In [ ]:
#seurat@meta.data[grep("TSC[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample <- gsub("TSC", "Day 4 Trophoblast", seurat@meta.data[grep("TSC[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample)
#seurat@meta.data[grep("TSCP", rownames(seurat@meta.data), perl=TRUE),]$sample <- gsub("TSCPRC2i", "Day 4 Trophoblast PRC2i", seurat@meta.data[grep("TSCP", rownames(seurat@meta.data), perl=TRUE),]$sample)
#seurat@meta.data[grep("NAIVE[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample <-  gsub("NAIVE", "Day 0 Naive hPSCs", seurat@meta.data[grep("NAIVE[^P]", rownames(seurat@meta.data), perl=TRUE),]$sample)
#seurat@meta.data[grep("NAIVEP", rownames(seurat@meta.data), perl=TRUE),]$sample <- gsub("NAIVEPRC2i", "Day 0 Naive hPSCs PRC2i", seurat@meta.data[grep("NAIVEP", rownames(seurat@meta.data), perl=TRUE),]$sample)
#head(seurat@meta.data)

### Check the sample column to make sure it only contains the names you want

In [50]:
#seurat@meta.data$sample %>% unique()

[1] "Day 4 Trophoblast"       "Day 4 Trophoblast PRC2i"
[3] "Day 0 Naive hPSCs"       "Day 0 Naive hPSCs PRC2i"

# Create loom file
#### This will fail if a loom file of the same name already exists

In [6]:
build_loom(file.name = "Talon_PRC2_integrated_fixed.loom",
        dgem = seurat@assays$RNA@counts,
        title = "Naive to trophoblast conversion with(out) PRC2 inhibition, integrated with embryo data",
        default.embedding = seurat@reductions$umap@cell.embeddings,
        default.embedding.name = "UMAP")

[1] "Adding global attributes..."
[1] "Adding matrix..."
  |======================================================================| 100%[1] "Adding column attributes..."
[1] "Adding default metrics nUMI..."
[1] "Adding default metrics nGene..."
[1] "Adding default embedding..."
[1] "Adding row attributes..."
[1] "Adding columns edges..."
[1] "Adding row edges..."
[1] "Adding layers..."


## Open the loom file for editing

In [22]:
#system("cp Irene.loom Irene.loom_bk")
#system("cp Irene.loom_bk Irene.loom")


#loom <- open_loom("Irene.loom")
loom <- open_loom(file.path = "Talon_PRC2_integrated_fixed.loom")

## Add two attribute columns
#### Key:

* name of the attribute

#### as.annotation = TRUE:

* discrete attribute

#### as.metric = TRUE:

* continuous attribute

In [8]:
seurat_meta.data_bk <- seurat@meta.data

In [9]:
seurat@meta.data[which(seurat@meta.data$seurat_clusters == 2), "Experiment"] %>% unique()

[1] "Zhou"                     "Pet"                     
[3] "naive_TSC_PRC2i_Irene_VP"

In [10]:
seurat@meta.data$finalClusters %>% unique()
seurat@meta.data$Experiment %>% unique()

[1] "TB.medium2"         NA                   "TB.late"           
 [4] "Pre.ST"             "TB.medium3"         "PrE"               
 [7] "Pre.EVT"            "EPI"                "ST"                
[10] "EVT"                "ysTE"               "early_TE"          
[13] "medium_TE"          "TB.medium1"         "TB.early"          
[16] "B1_B2"              "TB.apoptosis"       "late_TE"           
[19] "pre_implantation"   "Intermediate"       "TSCs"              
[22] "Naive"              "TSCs_PRC2i"         "Intermediate_PRC2i"

[1] "Zhou"                     "Pet"                     
[3] "naive_TSC_PRC2i_Irene_VP"

In [11]:
seurat@meta.data$Experiment <- gsub("naive_TSC_PRC2i_Irene_VP", "Talon", seurat@meta.data$Experiment)
seurat@meta.data$Experiment <- gsub("Pet", "Petropoulos", seurat@meta.data$Experiment)
seurat@meta.data$Experiment %>% unique()

[1] "Zhou"        "Petropoulos" "Talon"

In [12]:
seurat@meta.data$PRC2i_treatment <- NA
seurat@meta.data[which(seurat@meta.data$Experiment == "Talon"), "PRC2i_treatment"] <- "No"
seurat@meta.data[which(grepl("PRC2i", rownames(seurat@meta.data))), "PRC2i_treatment"] <- "Yes"

In [11]:
head(rownames(seurat@meta.data[which(seurat@meta.data$Experiment == "Talon"),]))

[1] "TSC_AAACCCACACTCCGGA.1" "TSC_AAACGAAGTAAGGCCA.1" "TSC_AAACGCTTCTACTGAG.1"
[4] "TSC_AAAGGGCCATGGAATA.1" "TSC_AAAGGTAAGATGCTGG.1" "TSC_AAAGTCCAGAAACACT.1"

In [13]:
seurat@meta.data$Castel_annotation <- seurat@meta.data$finalClusters
seurat@meta.data[which(seurat@meta.data$Experiment == "Talon"),"Castel_annotation"] <- "Unannotated"

In [14]:
seurat@meta.data$Talon_annotation <- NA
seurat@meta.data[rownames(data_IT@meta.data[which(data_IT@meta.data$seurat_clusters==0),]), "Talon_annotation"] <- "Naive"
seurat@meta.data[rownames(data_IT@meta.data[which(data_IT@meta.data$seurat_clusters==1),]), "Talon_annotation"] <- "Intermediate"
seurat@meta.data[rownames(data_IT@meta.data[which(data_IT@meta.data$seurat_clusters==2),]), "Talon_annotation"] <- "TSC"
seurat@meta.data$Talon_annotation %>% unique()

[1] NA             "Intermediate" "TSC"          "Naive"

In [15]:
data_IT@meta.data[data_IT@meta.data$seurat_clusters==0,]$cluster_PRC2i <- "Naive"
data_IT@meta.data[data_IT@meta.data$seurat_clusters==1,]$cluster_PRC2i <- "Intermediate"
data_IT@meta.data[data_IT@meta.data$seurat_clusters==2,]$cluster_PRC2i <- "TSCs"

Warning message in `[<-.data.frame`(`*tmp*`, data_IT@meta.data$seurat_clusters == :
"provided 8 variables to replace 7 variables"
Warning message in `[<-.data.frame`(`*tmp*`, data_IT@meta.data$seurat_clusters == :
"provided 8 variables to replace 7 variables"
Warning message in `[<-.data.frame`(`*tmp*`, data_IT@meta.data$seurat_clusters == :
"provided 8 variables to replace 7 variables"


In [ ]:
# Cols to add
# Talon_annotation
# PRC2i_treatment
# Experiment
# Castel_annotation
# Simplified_annotation (STILL TO COME)

In [17]:
data_simple_anno <- readRDS("/lustre1/project/stg_00041/Irene/10X_NAIVE_TSC_2021/nonaggr_noMEFs_mouse_human_analysis/Pet_Zhou_IT_allgenes_norm_new_anno.rds")

In [54]:
data_simple_anno@meta.data %>% dim()
data_simple_anno@meta.data %>% colnames()
data_IT@meta.data %>% dim()
seurat@meta.data %>% dim()
seurat@meta.data %>% colnames()

[1] 8809   19

[1] "orig.ident.x"        "nCount_RNA"          "nFeature_RNA"       
 [4] "percent.mt"          "percent.ERCC"        "Sample"             
 [7] "orig.ident.y"        "Experiment"          "combine_state"      
[10] "final_lineage"       "Cell.type"           "finalClustersold"   
[13] "Lineage"             "RNA_snn_res.0.1"     "seurat_clusters"    
[16] "percent.MT"          "RNA_snn_res.0.2"     "finalClusters"      
[19] "finalClustersMerged"

[1] 7629    7

[1] 8809   20

[1] "orig.ident.x"      "nCount_RNA"        "nFeature_RNA"     
 [4] "percent.mt"        "percent.ERCC"      "Sample"           
 [7] "orig.ident.y"      "Experiment"        "combine_state"    
[10] "final_lineage"     "Cell.type"         "finalClusters"    
[13] "Lineage"           "RNA_snn_res.0.1"   "seurat_clusters"  
[16] "percent.MT"        "RNA_snn_res.0.2"   "PRC2i_treatment"  
[19] "Castel_annotation" "Talon_annotation"

In [21]:
seurat@meta.data$Talon_annotation %>% unique()
seurat@meta.data$PRC2i_treatment %>% unique()
seurat@meta.data$Experiment %>% unique()
seurat@meta.data$Castel_annotation %>% unique()
seurat@meta.data$Simplified_annotation %>% unique()

[1] NA             "Intermediate" "TSC"          "Naive"

[1] NA    "No"  "Yes"

[1] "Zhou"        "Petropoulos" "Talon"

[1] "TB.medium2"       NA                 "TB.late"          "Pre.ST"          
 [5] "TB.medium3"       "PrE"              "Pre.EVT"          "EPI"             
 [9] "ST"               "EVT"              "ysTE"             "early_TE"        
[13] "medium_TE"        "TB.medium1"       "TB.early"         "B1_B2"           
[17] "TB.apoptosis"     "late_TE"          "pre_implantation" "Unannotated"

[1] "Embryo Trophoblast" "unknown"            "Embryo ST"         
 [4] "PrE"                "Embryo EVT"         "Epiblast"          
 [7] "ysTE"               "Embryo TE"          "(Pre) Blastocyst"  
[10] "EightCells"         "Intermediate"       "TSCs"              
[13] "Naive"              "TSCs_PRC2i"         "Intermediate_PRC2i"

In [23]:
add_col_attr(loom = loom, key = "Talon_annotation",
              value = seurat@meta.data$Talon_annotation, as.annotation = TRUE)
add_col_attr(loom = loom, key = "PRC2i_treatment",
              value = seurat@meta.data$PRC2i_treatment, as.annotation = TRUE)
add_col_attr(loom = loom, key = "Experiment",
              value = seurat@meta.data$Experiment, as.annotation = TRUE)
add_col_attr(loom = loom, key = "Castel_annotation",
              value = seurat@meta.data$Castel_annotation, as.annotation = TRUE)
add_col_attr(loom = loom, key = "Simplified_annotation",
              value = seurat@meta.data$Simplified_annotation, as.annotation = TRUE)

In [52]:
seurat_meta.data_backup <- seurat@meta.data

In [20]:
seurat@meta.data <- seurat@meta.data[which(!grepl("NAIVE", rownames(seurat@meta.data))),]

In [55]:
seurat_meta.data_backup_no_naive <- seurat@meta.data

In [19]:
seurat@meta.data[rownames(data_simple_anno@meta.data), "Simplified_annotation"] <- data_simple_anno@meta.data$finalClustersMerged 

In [57]:
head(seurat@meta.data)

,orig.ident.x,nCount_RNA,nFeature_RNA,percent.mt,percent.ERCC,Sample,orig.ident.y,Experiment,combine_state,final_lineage,⋯,finalClusters,Lineage,RNA_snn_res.0.1,seurat_clusters,percent.MT,RNA_snn_res.0.2,PRC2i_treatment,Castel_annotation,Talon_annotation,Simplified_annotation
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
D10_IVC4_E1_B1_10,VP_Messmer_Petropoulos_Zhou,788590,10072,0,0,D10_IVC4_E1_B1_10,IVC4,Zhou,E1,TE,⋯,TB.medium2,TE _ Zhou,0,0,NA,NA,No,TB.medium2,NA,Embryo Trophoblast
D10_IVC4_E1_B1_11,VP_Messmer_Petropoulos_Zhou,46029,3923,0,0,D10_IVC4_E1_B1_11,IVC4,Zhou,E1,TE,⋯,NA,TE _ Zhou,1,1,NA,NA,No,NA,NA,unknown
D10_IVC4_E1_B1_12,VP_Messmer_Petropoulos_Zhou,146063,7508,0,0,D10_IVC4_E1_B1_12,IVC4,Zhou,E1,TE,⋯,TB.late,TE _ Zhou,0,0,NA,NA,No,TB.late,NA,Embryo Trophoblast
D10_IVC4_E1_B1_13,VP_Messmer_Petropoulos_Zhou,1262641,10042,0,0,D10_IVC4_E1_B1_13,IVC4,Zhou,E1,TE,⋯,Pre.ST,TE _ Zhou,1,1,NA,NA,No,Pre.ST,NA,Embryo ST
D10_IVC4_E1_B1_14,VP_Messmer_Petropoulos_Zhou,24702,3347,0,0,D10_IVC4_E1_B1_14,IVC4,Zhou,E1,TE,⋯,NA,TE _ Zhou,1,1,NA,NA,No,NA,NA,unknown
D10_IVC4_E1_B1_15,VP_Messmer_Petropoulos_Zhou,2358380,12107,0,0,D10_IVC4_E1_B1_15,IVC4,Zhou,E1,TE,⋯,TB.late,TE _ Zhou,1,1,NA,NA,No,TB.late,NA,Embryo Trophoblast


# Done!
#### Loom file is automatically updated

In [ ]:
sessionInfo()